In [14]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import roc_auc_score, confusion_matrix

In [15]:
#Load data
df = pd.read_csv("EEG.machinelearing_data_BRMH.csv")

#Define variables and number of samples
bands = {'delta','theta','alpha','beta','highbeta','whole'}
disorders = ['Alcohol use disorder', 'Behavioral addiction disorder']
combos = ['PSD', 'FC', 'PSD+FC']
N_samples = 10

psd_columns = [col for col in df.columns if col.startswith('AB.')]
fc_columns = [col for col in df.columns if col.startswith('COH.')]

#Function to extract psd and fc values
def extract_psd(band):
    return [col for col in psd_columns if f'.{band}.' in col]

def extract_fc(band):
    return [col for col in fc_columns if f'.{band}.' in col]

Best_Feature = []

for disorder in disorders:
    print(disorder)

    disorder_patients = df[df['specific.disorder'] == disorder].sample(
        n=min(N_samples, len(df[df['specific.disorder'] == disorder])),
        random_state=42
    )
    healthy_controls = df[df['specific.disorder'] == 'Healthy control'].sample(
        n=min(N_samples, len(df[df['specific.disorder'] == 'Healthy control'])),
        random_state=42
    )
    df_sub = pd.concat([disorder_patients, healthy_controls])
    df_sub['binary_label'] = df_sub['specific.disorder'].apply(lambda x: 1 if x == disorder else 0)
    unique_classes, class_counts = np.unique(df_sub['binary_label'], return_counts=True)
    if len(unique_classes) < 2 or np.any(class_counts < 2):
        print(f"Not enough samples ({dict(zip(unique_classes, class_counts))})")
        continue

    best_auc = -1
    best_result = None

    #Loop through bands
    for band in bands:
        if band == 'whole':
            band_psd_cols = psd_columns
            band_fc_cols = fc_columns
        else:
            band_psd_cols = extract_psd(band)
            band_fc_cols = extract_fc(band)

        #Loop PSD/FC/Both
        for combo in combos:
            # Select feature columns
            if combo == 'PSD':
                feature_cols = band_psd_cols
            elif combo == 'FC':
                feature_cols = band_fc_cols
            elif combo == 'PSD+FC':
                feature_cols = band_psd_cols + band_fc_cols

            # Skip empty combos
            if len(feature_cols) == 0:
                continue

            X = df_sub[feature_cols].copy()
            X = X.fillna(X.mean())
            y = df_sub['binary_label'].values
            
            cv = StratifiedShuffleSplit(n_splits=10, test_size=0.3, random_state=42)

            auc_list = []
            sens_list = []
            spec_list = []

            for train_idx, test_idx in cv.split(X, y):
                X_train, y_train = X.iloc[train_idx], y[train_idx]
                X_test, y_test = X.iloc[test_idx], y[test_idx]

                pipeline = Pipeline([
                    ('scaler', StandardScaler()),
                    ('clf', LogisticRegression(
                        penalty='elasticnet',
                        l1_ratio=0.5,
                        solver='saga',
                        max_iter=5000,
                        random_state=42
                    ))
                ])

                # Fit
                pipeline.fit(X_train, y_train)

                # Predict
                y_prob = pipeline.predict_proba(X_test)[:, 1]
                y_pred = pipeline.predict(X_test)

                # AUC
                auc = roc_auc_score(y_test, y_prob)
                auc_list.append(auc)

                # Confusion matrix
                cm = confusion_matrix(y_test, y_pred)
                tn, fp, fn, tp = cm.ravel()

                sens = tp / (tp + fn) if (tp + fn) > 0 else 0.0
                spec = tn / (tn + fp) if (tn + fp) > 0 else 0.0

                sens_list.append(sens)
                spec_list.append(spec)

            # Aggregate results
            auc_mean = np.mean(auc_list) * 100
            auc_std = np.std(auc_list) * 100

            sens_mean = np.mean(sens_list) * 100
            sens_std = np.std(sens_list) * 100

            spec_mean = np.mean(spec_list) * 100
            spec_std = np.std(spec_list) * 100

            print(f"{disorder} Band: {band} Combo: {combo}  AUC: {auc_mean:.2f} ({auc_std:.2f})")

            result = {
                'disorder': disorder,
                'band': band,
                'combo': combo,
                'AUC_mean': auc_mean,
                'AUC_std': auc_std,
                'Sens_mean': sens_mean,
                'Sens_std': sens_std,
                'Spec_mean': spec_mean,
                'Spec_std': spec_std
            }

            Best_Feature.append(result)

            if auc_mean > best_auc:
                best_auc = auc_mean
                best_result = result
    print(f"Best model for {disorder}")
    print(f"Feature: {best_result['band']} {best_result['combo']}")
    print(f"AUC: {best_result['AUC_mean']:.2f} ({best_result['AUC_std']:.2f})")
    print(f"Sens: {best_result['Sens_mean']:.2f} ({best_result['Sens_std']:.2f})")
    print(f"Spec: {best_result['Spec_mean']:.2f} ({best_result['Spec_std']:.2f})")

BRMH_df = pd.DataFrame(Best_Feature)
BRMH_df = BRMH_df.sort_values(by=['disorder', 'AUC_mean'], ascending=[True, False])
BRMH_df.to_csv("BRMH_Best_Features.csv", index=False)


Alcohol use disorder
Alcohol use disorder Band: highbeta Combo: PSD  AUC: 35.56 (23.73)
Alcohol use disorder Band: highbeta Combo: FC  AUC: 74.44 (18.63)
Alcohol use disorder Band: highbeta Combo: PSD+FC  AUC: 68.89 (19.12)
Alcohol use disorder Band: beta Combo: PSD  AUC: 23.33 (17.53)
Alcohol use disorder Band: beta Combo: FC  AUC: 71.11 (20.61)
Alcohol use disorder Band: beta Combo: PSD+FC  AUC: 66.67 (24.85)
Alcohol use disorder Band: alpha Combo: PSD  AUC: 34.44 (23.54)
Alcohol use disorder Band: alpha Combo: FC  AUC: 57.78 (22.66)
Alcohol use disorder Band: alpha Combo: PSD+FC  AUC: 51.11 (23.93)
Alcohol use disorder Band: delta Combo: PSD  AUC: 52.22 (17.95)
Alcohol use disorder Band: delta Combo: FC  AUC: 58.89 (12.22)
Alcohol use disorder Band: delta Combo: PSD+FC  AUC: 50.00 (23.44)
Alcohol use disorder Band: theta Combo: PSD  AUC: 23.33 (23.01)
Alcohol use disorder Band: theta Combo: FC  AUC: 65.56 (16.81)
Alcohol use disorder Band: theta Combo: PSD+FC  AUC: 61.11 (23.44)
Alc

In [16]:
# SETTINGS
stable_bands = {'delta','theta','alpha','beta','highbeta'}
stable_combos = ['PSD+FC']
N_samples = 10

#Loop over disorders, bands, and feature combinations
for disorder in disorders:
    for band in stable_bands:
        for combo in stable_combos:
            print(f" {disorder}  Band: {band}  Combo: {combo} ===")

            # Subset data
            disorder_patients = df[df['specific.disorder'] == disorder].sample(
                n=min(N_samples, len(df[df['specific.disorder'] == disorder])),
                random_state=42
            )
            healthy_controls = df[df['specific.disorder'] == 'Healthy control'].sample(
                n=min(N_samples, len(df[df['specific.disorder'] == 'Healthy control'])),
                random_state=42
            )
            df_sub = pd.concat([disorder_patients, healthy_controls])
            df_sub['binary_label'] = df_sub['specific.disorder'].apply(lambda x: 1 if x == disorder else 0)

            # Feature selection
            band_psd_cols = extract_psd(band)
            band_fc_cols = extract_fc(band)
            feature_cols = band_psd_cols + band_fc_cols

            if len(feature_cols) < 2:
                continue

            # Prepare X, y
            X = df_sub[feature_cols].copy().fillna(df_sub[feature_cols].mean())
            y = df_sub['binary_label'].values

            unique_classes, class_counts = np.unique(y, return_counts=True)
            if len(unique_classes) < 2 or np.any(class_counts < 2):
                continue

            # Manual CV loop to collect coefficients
            cv = StratifiedShuffleSplit(n_splits=10, test_size=0.3, random_state=42)
            coef_matrix = []

            for train_idx, test_idx in cv.split(X, y):
                X_train, y_train = X.iloc[train_idx], y[train_idx]
                pipeline = Pipeline([
                    ('scaler', StandardScaler()),
                    ('clf', LogisticRegression(
                        penalty='elasticnet',
                        l1_ratio=0.5,
                        solver='saga',
                        max_iter=5000,
                        random_state=42
                    ))
                ])
                pipeline.fit(X_train, y_train)
                coefs = pipeline.named_steps['clf'].coef_[0]
                coef_matrix.append(coefs)

            # Analyze coefficients
            coef_matrix = np.array(coef_matrix)
            coef_df = pd.DataFrame(coef_matrix, columns=feature_cols)

            results = []
            for feature in coef_df.columns:
                non_zero_count = (coef_df[feature] != 0).sum()
                mean_coef = coef_df[feature].mean()
                results.append({
                    'feature': feature,
                    'n_selected_folds': non_zero_count,
                    'mean_coef': mean_coef
                })

            results_df = pd.DataFrame(results)
            results_df = results_df.sort_values(by='n_selected_folds', ascending=False)

            # Display and save stable features
            output_folder = os.path.join(os.getcwd(), "BRMH_stable_feature_outputs")
            os.makedirs(output_folder, exist_ok=True)
            stable_features_df = results_df[results_df['n_selected_folds'] >= 7]
            print(f"Stable features (≥7 folds): {len(stable_features_df)}")

            output_filename = f"BRMH_EN_stable_features_{disorder.replace(' ', '_')}_{band}_{combo}.csv"
            output_path = os.path.join(output_folder, output_filename)
            stable_features_df.to_csv(output_path, index=False)

 Alcohol use disorder  Band: highbeta  Combo: PSD+FC ===
Stable features (≥7 folds): 10
 Alcohol use disorder  Band: beta  Combo: PSD+FC ===
Stable features (≥7 folds): 7
 Alcohol use disorder  Band: delta  Combo: PSD+FC ===
Stable features (≥7 folds): 8
 Alcohol use disorder  Band: theta  Combo: PSD+FC ===
Stable features (≥7 folds): 11
 Alcohol use disorder  Band: alpha  Combo: PSD+FC ===
Stable features (≥7 folds): 9
 Behavioral addiction disorder  Band: highbeta  Combo: PSD+FC ===
Stable features (≥7 folds): 10
 Behavioral addiction disorder  Band: beta  Combo: PSD+FC ===
Stable features (≥7 folds): 5
 Behavioral addiction disorder  Band: delta  Combo: PSD+FC ===
Stable features (≥7 folds): 10
 Behavioral addiction disorder  Band: theta  Combo: PSD+FC ===
Stable features (≥7 folds): 9
 Behavioral addiction disorder  Band: alpha  Combo: PSD+FC ===
Stable features (≥7 folds): 8


In [17]:
#Load data
df = pd.read_csv("LEMON_EEG.csv")

#Define variables and number of samples
bands = {'delta','theta','alpha','beta','highbeta','whole'}
disorders = ['Alcohol use disorder', 'Behavioral addiction disorder']
combos = ['PSD', 'FC', 'PSD+FC']
N_samples = 10

psd_columns = [col for col in df.columns if col.startswith('AB.')]
fc_columns = [col for col in df.columns if col.startswith('COH.')]

#Function to extract psd and fc values
def extract_psd(band):
    return [col for col in psd_columns if f'.{band}.' in col]

def extract_fc(band):
    return [col for col in fc_columns if f'.{band}.' in col]

Best_Feature = []

for disorder in disorders:
    print(disorder)

    disorder_patients = df[df['specific.disorder'] == disorder].sample(
        n=min(N_samples, len(df[df['specific.disorder'] == disorder])),
        random_state=42
    )
    healthy_controls = df[df['specific.disorder'] == 'Healthy control'].sample(
        n=min(N_samples, len(df[df['specific.disorder'] == 'Healthy control'])),
        random_state=42
    )
    df_sub = pd.concat([disorder_patients, healthy_controls])
    df_sub['binary_label'] = df_sub['specific.disorder'].apply(lambda x: 1 if x == disorder else 0)
    unique_classes, class_counts = np.unique(df_sub['binary_label'], return_counts=True)
    if len(unique_classes) < 2 or np.any(class_counts < 2):
        print(f"Not enough samples ({dict(zip(unique_classes, class_counts))})")
        continue

    best_auc = -1
    best_result = None

    #Loop through bands
    for band in bands:
        if band == 'whole':
            band_psd_cols = psd_columns
            band_fc_cols = fc_columns
        else:
            band_psd_cols = extract_psd(band)
            band_fc_cols = extract_fc(band)

        #Loop PSD/FC/Both
        for combo in combos:
            # Select feature columns
            if combo == 'PSD':
                feature_cols = band_psd_cols
            elif combo == 'FC':
                feature_cols = band_fc_cols
            elif combo == 'PSD+FC':
                feature_cols = band_psd_cols + band_fc_cols

            # Skip empty combos
            if len(feature_cols) == 0:
                continue

            X = df_sub[feature_cols].copy()
            X = X.fillna(X.mean())
            y = df_sub['binary_label'].values
            
            cv = StratifiedShuffleSplit(n_splits=10, test_size=0.3, random_state=42)

            auc_list = []
            sens_list = []
            spec_list = []

            for train_idx, test_idx in cv.split(X, y):
                X_train, y_train = X.iloc[train_idx], y[train_idx]
                X_test, y_test = X.iloc[test_idx], y[test_idx]

                pipeline = Pipeline([
                    ('scaler', StandardScaler()),
                    ('clf', LogisticRegression(
                        penalty='elasticnet',
                        l1_ratio=0.5,
                        solver='saga',
                        max_iter=5000,
                        random_state=42
                    ))
                ])

                # Fit
                pipeline.fit(X_train, y_train)

                # Predict
                y_prob = pipeline.predict_proba(X_test)[:, 1]
                y_pred = pipeline.predict(X_test)

                # AUC
                auc = roc_auc_score(y_test, y_prob)
                auc_list.append(auc)

                # Confusion matrix
                cm = confusion_matrix(y_test, y_pred)
                tn, fp, fn, tp = cm.ravel()

                sens = tp / (tp + fn) if (tp + fn) > 0 else 0.0
                spec = tn / (tn + fp) if (tn + fp) > 0 else 0.0

                sens_list.append(sens)
                spec_list.append(spec)

            # Aggregate results
            auc_mean = np.mean(auc_list) * 100
            auc_std = np.std(auc_list) * 100

            sens_mean = np.mean(sens_list) * 100
            sens_std = np.std(sens_list) * 100

            spec_mean = np.mean(spec_list) * 100
            spec_std = np.std(spec_list) * 100

            print(f"{disorder} Band: {band} Combo: {combo}  AUC: {auc_mean:.2f} ({auc_std:.2f})")

            result = {
                'disorder': disorder,
                'band': band,
                'combo': combo,
                'AUC_mean': auc_mean,
                'AUC_std': auc_std,
                'Sens_mean': sens_mean,
                'Sens_std': sens_std,
                'Spec_mean': spec_mean,
                'Spec_std': spec_std
            }

            Best_Feature.append(result)

            if auc_mean > best_auc:
                best_auc = auc_mean
                best_result = result
    print(f"Best model for {disorder}")
    print(f"Feature: {best_result['band']} {best_result['combo']}")
    print(f"AUC: {best_result['AUC_mean']:.2f} ({best_result['AUC_std']:.2f})")
    print(f"Sens: {best_result['Sens_mean']:.2f} ({best_result['Sens_std']:.2f})")
    print(f"Spec: {best_result['Spec_mean']:.2f} ({best_result['Spec_std']:.2f})")

BRMH_df = pd.DataFrame(Best_Feature)
BRMH_df = BRMH_df.sort_values(by=['disorder', 'AUC_mean'], ascending=[True, False])
BRMH_df.to_csv("LEMON_Best_Features.csv", index=False)


Alcohol use disorder
Alcohol use disorder Band: beta Combo: FC  AUC: 68.33 (26.30)
Alcohol use disorder Band: beta Combo: PSD+FC  AUC: 68.33 (26.30)
Alcohol use disorder Band: alpha Combo: FC  AUC: 58.33 (8.33)
Alcohol use disorder Band: alpha Combo: PSD+FC  AUC: 58.33 (8.33)
Alcohol use disorder Band: delta Combo: FC  AUC: 50.00 (0.00)
Alcohol use disorder Band: delta Combo: PSD+FC  AUC: 50.00 (0.00)
Alcohol use disorder Band: theta Combo: FC  AUC: 50.00 (0.00)
Alcohol use disorder Band: theta Combo: PSD+FC  AUC: 50.00 (0.00)


/home/jerome/miniconda3/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jerome/miniconda3/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jerome/miniconda3/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jerome/miniconda3/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jerome/miniconda3/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jerome/miniconda3/lib/python3.13/site-packages/sklearn

Alcohol use disorder Band: whole Combo: FC  AUC: 58.33 (25.00)


/home/jerome/miniconda3/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jerome/miniconda3/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jerome/miniconda3/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jerome/miniconda3/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jerome/miniconda3/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jerome/miniconda3/lib/python3.13/site-packages/sklearn

Alcohol use disorder Band: whole Combo: PSD+FC  AUC: 58.33 (25.00)
Best model for Alcohol use disorder
Feature: beta FC
AUC: 68.33 (26.30)
Sens: 65.00 (30.23)
Spec: 51.67 (45.00)
Behavioral addiction disorder
Not enough samples ({np.int64(0): np.int64(8)})


In [18]:
# SETTINGS
stable_bands = {'delta','theta','alpha','beta','highbeta'}
stable_combos = ['PSD+FC']
N_samples = 10

#Loop over disorders, bands, and feature combinations
for disorder in disorders:
    for band in stable_bands:
        for combo in stable_combos:
            print(f" {disorder}  Band: {band}  Combo: {combo} ===")

            # Subset data
            disorder_patients = df[df['specific.disorder'] == disorder].sample(
                n=min(N_samples, len(df[df['specific.disorder'] == disorder])),
                random_state=42
            )
            healthy_controls = df[df['specific.disorder'] == 'Healthy control'].sample(
                n=min(N_samples, len(df[df['specific.disorder'] == 'Healthy control'])),
                random_state=42
            )
            df_sub = pd.concat([disorder_patients, healthy_controls])
            df_sub['binary_label'] = df_sub['specific.disorder'].apply(lambda x: 1 if x == disorder else 0)

            # Feature selection
            band_psd_cols = extract_psd(band)
            band_fc_cols = extract_fc(band)
            feature_cols = band_psd_cols + band_fc_cols

            if len(feature_cols) < 2:
                continue

            # Prepare X, y
            X = df_sub[feature_cols].copy().fillna(df_sub[feature_cols].mean())
            y = df_sub['binary_label'].values

            unique_classes, class_counts = np.unique(y, return_counts=True)
            if len(unique_classes) < 2 or np.any(class_counts < 2):
                continue

            # Manual CV loop to collect coefficients
            cv = StratifiedShuffleSplit(n_splits=10, test_size=0.3, random_state=42)
            coef_matrix = []

            for train_idx, test_idx in cv.split(X, y):
                X_train, y_train = X.iloc[train_idx], y[train_idx]
                pipeline = Pipeline([
                    ('scaler', StandardScaler()),
                    ('clf', LogisticRegression(
                        penalty='elasticnet',
                        l1_ratio=0.5,
                        solver='saga',
                        max_iter=5000,
                        random_state=42
                    ))
                ])
                pipeline.fit(X_train, y_train)
                coefs = pipeline.named_steps['clf'].coef_[0]
                coef_matrix.append(coefs)

            # Analyze coefficients
            coef_matrix = np.array(coef_matrix)
            coef_df = pd.DataFrame(coef_matrix, columns=feature_cols)

            results = []
            for feature in coef_df.columns:
                non_zero_count = (coef_df[feature] != 0).sum()
                mean_coef = coef_df[feature].mean()
                results.append({
                    'feature': feature,
                    'n_selected_folds': non_zero_count,
                    'mean_coef': mean_coef
                })

            results_df = pd.DataFrame(results)
            results_df = results_df.sort_values(by='n_selected_folds', ascending=False)

            # Display and save stable features
            output_folder = os.path.join(os.getcwd(), "LEMON_stable_feature_outputs")
            os.makedirs(output_folder, exist_ok=True)
            stable_features_df = results_df[results_df['n_selected_folds'] >= 7]
            print(f"Stable features (≥7 folds): {len(stable_features_df)}")

            output_filename = f"LEMON_EN_stable_features_{disorder.replace(' ', '_')}_{band}_{combo}.csv"
            output_path = os.path.join(output_folder, output_filename)
            stable_features_df.to_csv(output_path, index=False)

 Alcohol use disorder  Band: highbeta  Combo: PSD+FC ===
 Alcohol use disorder  Band: beta  Combo: PSD+FC ===
Stable features (≥7 folds): 9
 Alcohol use disorder  Band: delta  Combo: PSD+FC ===
Stable features (≥7 folds): 1891
 Alcohol use disorder  Band: theta  Combo: PSD+FC ===
Stable features (≥7 folds): 1891
 Alcohol use disorder  Band: alpha  Combo: PSD+FC ===
Stable features (≥7 folds): 15
 Behavioral addiction disorder  Band: highbeta  Combo: PSD+FC ===
 Behavioral addiction disorder  Band: beta  Combo: PSD+FC ===
 Behavioral addiction disorder  Band: delta  Combo: PSD+FC ===
 Behavioral addiction disorder  Band: theta  Combo: PSD+FC ===
 Behavioral addiction disorder  Band: alpha  Combo: PSD+FC ===
